In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
#set up the URL for scraping
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
#we'll assign the url and instruct the browser to visit it.
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
# we're searching for elements with a specific combination of tag (ul and li) and attribute (item_list and slide, respectively)
#we're also telling our browser to wait one second before searching for components.
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
#set up the HTML parser:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# After searching for the HTML components you'll use to identify the title and paragraph you want
#Begin scraping
#The specific data is in a <div /> with a class of 'content_title'
slide_elem.find("div", class_='content_title')


<div class="content_title"><a href="/news/8782/sensors-on-mars-2020-spacecraft-answer-long-distance-call-from-earth/" target="_self">Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth</a></div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth'

In [7]:
#There are two methods used to find tags and attributes with BeautifulSoup:

#.find() is used when we want only the first class and attribute we've specified.
#.find_all() is used when we want to retrieve all of the tags and attributes.

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p


"Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."

### Featured Images

In [9]:
#Let's start getting our code ready to automate all of the clicks to get the image we want
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [11]:
# Find the more info button and click that

#method to search for an element that has the provided text
#wait_time allows the browser to fully load before we search for the element
browser.is_element_present_by_text('more info', wait_time=1)
#This method will take our string ‘more info’ to find the link associated with the "more info" text.
more_info_elem = browser.links.find_by_partial_text('more info')
#tell Splinter to click that link
more_info_elem.click()

#browser will navigate to the next page.

In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [13]:
# Find the relative image url
# "This is where the image we want lives—use the link that's inside these tags."
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA16225_hires.jpg'

In [14]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

In [15]:
#scrape the entire table with Pandas'
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
#the code is designed to grab the most recent data, so if it's run at a later time, all of the results will have been updated